# Explingo Experiment Runner

This notebook:
1. Loads the gold-standard dataset, prepares the metrics functions, and verifies that the metric functions give the maximum score on the gold-standard dataset, and lower scores on less aligned datasets
2. Runs the prompt-design, few-shot, and bootstrap-few-shot experiments on a testing dataset

## Setup
Import necessary libraries and prepare the LLM

**Note: To run these cells, you need a `keys.yaml` file in the top-level Explingo directory with the following line:**
```yaml
openai_api_key: <your_openai_api_key>
```

In [1]:
import pandas as pd

from experiment_runner import ExplingoExperimentRunner
import os
import yaml
import dspy
import metrics
import random

In [2]:
with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

llm = dspy.OpenAI(model='gpt-4o', api_key=openai_api_key, max_tokens=1000)

Now, we create the main experiment runner object. This object takes in a dataset, and then
1. Splits the dataset into a training dataset and a testing dataset (see notes below)
2. Sets up the evaluation metrics (see notes below). The fluency metric is set up to use sample from the dataset as reference
3. Runs the experiments on the testing dataset

Some examples in the testing datasets include gold-standard narratives; others include only a sample explanation.
- The former makes up the gold-standard dataset used for tuning the evaluation metrics and providing few-shot examples.
- The latter makes up the testing dataset used for evaluation and for bootstrapping few-shot examples

We use the following metrics, all scored on a scale from 0-4:
- Accuracy: the narrative accurately describes the information in the explanation
- Fluency: the narrative is coherent and natural, as compared to the gold-standard explanations. We pass in a small list of sample narratives from the gold-standard dataset to compare against
- Conciseness: the narrative is not too long, as compared to the gold-standard explanations. For now, any narrative that is no longer than the longest gold-standard narrative will score 4
- Completeness: the narrative includes all relevant information from the original explanation 

**Note: You can set `verbose=1` to see the narratives generated, or `verbose=2` to see the explanations, narratives, and rationalizations**

In [3]:
# iterate all datasets in the eval_data folder
runners = {}
total_eval = 0
for dataset in os.listdir(os.path.join("eval_data")):
    runners[dataset] = ExplingoExperimentRunner(llm=llm, openai_api_key=openai_api_key, dataset_filepath = os.path.join("eval_data", dataset), verbose=1)
    total_eval += len(runners[dataset].eval_data)
    
print("Total eval examples:", total_eval)
results = []

eval_data\housing_1.json
Total number of examples: 35
Labeled training examples: 5
Labeled evaluation examples: 15
Unlabeled training examples: 5
Unlabeled evaluation examples: 10
---
Max optimal length: 54.400000000000006
eval_data\housing_2.json
Total number of examples: 22
Labeled training examples: 5
Labeled evaluation examples: 7
Unlabeled training examples: 5
Unlabeled evaluation examples: 5
---
Max optimal length: 40.0
eval_data\housing_3.json
Total number of examples: 22
Labeled training examples: 5
Labeled evaluation examples: 8
Unlabeled training examples: 5
Unlabeled evaluation examples: 4
---
Max optimal length: 30.400000000000002
eval_data\mushroom_1.json
Total number of examples: 30
Labeled training examples: 5
Labeled evaluation examples: 6
Unlabeled training examples: 5
Unlabeled evaluation examples: 14
---
Max optimal length: 26.400000000000002
eval_data\mushroom_2.json
Total number of examples: 30
Labeled training examples: 5
Labeled evaluation examples: 6
Unlabeled t

## Basic prompt design experiment

We begin with basic prompts. With 4 metrics (without completeness), each with a score of 0-2, the maximum score is 8. 

We generate narratives/rationalizations on `max_iters=5` sample explanations, and return the average total score.

In [4]:
# Utilities for cleaner results

def pretty_print(result):
    s = f"Total score: {result[0]}"
    s2 = ", ".join([f"{k}: {v}" for k, v in result[1].items()])
    print(f"{s} ({s2})")
    
def update_results(method, dataset, scores, kwargs):
    result = {"dataset": dataset, "prompt": prompt, "total score": scores[0]}
    result.update(scores[1])
    result.update(kwargs)
    results.append(result)

In [5]:
prompts = ["You are helping users understand an ML model's prediction. Given an explanation and information about the model, "
           "convert the explanation into a human-readable narrative.",
           "You are helping users who do not have experience working with ML understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Make your answers sound as natural as possible.",
           "You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative. Be sure to explicitly mention all values from the explanation in your response.",
]

for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for prompt in prompts:
        print(f"Prompt: {prompt}")
        scores = runner.run_basic_prompting_experiment(prompt=prompt, max_iters=5)
        update_results("basic_prompting", dataset, scores, {"prompt": prompt})
        pretty_print(scores)
        print("--")
    print("=====")

Dataset: housing_1.json
Prompt: You are helping users understand an ML model's prediction. Given an explanation and information about the model, convert the explanation into a human-readable narrative.
Narrative: The model predicts the house price based on several key factors. The house is located within the Ames city limits, specifically in the NoRidge neighborhood, which contributes significantly to the price with a value of 23069.89. The above ground living area is 2198 square feet, adding 20125.75 to the price. The second floor has 1053 square feet, contributing 18094.05. The overall material and finish of the house are rated at 8, which adds 9655.79 to the price. Lastly, the house was originally constructed in the year 2000, contributing 8192.46 to the final predicted price.
Total Score: 1.0882352941176476
conciseness: 1.0882352941176476, 
--
Narrative: The model predicts the house price based on several factors. The type of foundation being wood decreases the price by $18,650.67.

## Few-shot experiment

Next, we repeat the experiment with the addition of N few-shot examples from the gold-standard dataset.

In [6]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    for i in [1, 3, 5]:
        print(f"Few-shot n: {i}")
        scores = runner.run_few_shot_experiment(n_few_shot=i, prompt=prompts[0], max_iters=5)
        update_results("few_shot", dataset, scores, {"n_few_shot": i, "prompt": prompts[0]})
        pretty_print(scores)
        print("--")
    print("=====")

Dataset: housing_1.json
Few-shot n: 1
Narrative: The house's location in NoRidge increased the predicted price by about $23,000. The relatively larger above ground living space increased the price by about $20,000. The house's larger than average second floor increased the price by about $18,000. The good quality of materials and finishes (rated 8/10) increased the price by about $9,500. The house is newer than average, with a construction year of 2000, which increased the price by about $8,000.
Total Score: 2.7058823529411766
conciseness: 2.7058823529411766, 
--
Narrative: The house's wood foundation reduced the predicted price by about $19,000. The house's location in Mitchel reduced the price by about $13,500. The overall material and finish rating of 5 reduced the price by about $10,500. The presence of a three-season porch increased the price by about $10,000. Having one bedroom above ground increased the price by about $9,000.
Total Score: 3.6617647058823533
conciseness: 3.661764

KeyboardInterrupt: 

## Bootstrapped few-shot
Next, we repeat the experiment with the addition of 3 examples bootstrapped by DSPy to optimize the evaluation metrics.

In [ ]:
for dataset in runners:
    runner = runners[dataset]
    print(f"Dataset: {dataset}")
    #for i, j in [[0, 1], [0, 3], [3, 3]]:
    for i, j in [[3, 3]]:
        print(f"Few-shot n: {i}, Bootstrapped n: {j}")
        scores = runner.run_bootstrap_few_shot_experiment(n_labeled_few_shot=i, n_bootstrapped_few_shot=j, max_iters=5)
        update_results("bootstrap_few_shot", dataset, scores, {"n_few_shot": i, "n_bootstrapped_few_shot": j, "prompt": prompts[0]})
        pretty_print(scores)
        print("--")
    print("=====")

In [ ]:
llm.inspect_history(n=1)

In [ ]:
result_df = pd.DataFrame(results)
result_df.to_csv("results.csv")
result_df